In [4]:
import numpy as np
import tensorflow as tf
from copy import deepcopy

import import_ipynb
import QBert

import pickle
from tqdm.notebook import tqdm

def load_pkl(file_path) :
    
    with open(file_path, 'rb') as f:
        df = pickle.load(f)
        
    return df

def save_pkl(df) :
    
    with open(file_path, 'wb') as f:
        pickle.dump(df, f)
        